# World Health Statistics
### Merged according to category similarity

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import warnings
from itertools import product
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore", ConvergenceWarning)

In [2]:
# Define the directory path
directory = r"D:\IE 423\Project\ie423-2024-termproject-the-a-team\ie423-2024-termproject-the-a-team\DataSets\WorldHealth"

# Define the filenames for each group of CSV files
mortality_files = ["infantMortalityRate.csv", "under5MortalityRate.csv", "neonatalMortalityRate.csv", "mortalityRatePoisoning.csv", "crudeSuicideRates.csv", "30-70cancerChdEtc.csv"]
life_expectancy_files = ["lifeExpectancyAtBirth.csv", "WHOregionLifeExpectancyAtBirth.csv", "HALeWHOregionLifeExpectancyAtBirth.csv", "HALElifeExpectancyAtBirth.csv", "%HaleInLifeExpectancy.csv"]
health_services_files = ["basicDrinkingWaterServices.csv", "atLeastBasicSanitizationServices.csv", "safelySanitization.csv", "basicHandWashing.csv", "birthAttendedBySkilledPersonal.csv"]
disease_incidence_files = ["interventionAgianstNTDs.csv", "incedenceOfMalaria.csv", "incedenceOfTuberculosis.csv", "hepatitusBsurfaceAntigen.csv", "newHivInfections.csv"]
maternal_health_files = ["maternalMortalityRatio.csv"]

# Function to merge CSV files
def merge_csv_files(files):
    dfs = []
    for file in files:
        filepath = os.path.join(directory, file)
        if os.path.exists(filepath):
            df = pd.read_csv(filepath)
            dfs.append(df)
        else:
            print(f"File '{file}' not found.")
    if dfs:
        merged_df = pd.concat(dfs, ignore_index=True)
        return merged_df
    else:
        return None

# Merge mortality files
merged_mortality = merge_csv_files(mortality_files)

# Merge life expectancy files
merged_life_expectancy = merge_csv_files(life_expectancy_files)

# Merge health services files
merged_health_services = merge_csv_files(health_services_files)

# Merge disease incidence files
merged_disease_incidence = merge_csv_files(disease_incidence_files)

# Merge maternal health files
merged_maternal_health = merge_csv_files(maternal_health_files)


In [3]:
# List all CSV files in the directory
csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

# Loop through each CSV file
for file in csv_files:
    # Construct the full path to the CSV file
    file_path = os.path.join(directory, file)
    
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Display basic information about the DataFrame
    print(f"Information about {file}:")
    print(df.info())
    print("\nSummary statistics:")
    print(df.describe())
    print("\nSample data:")
    print(df.head())
    print("\n\n")


Information about %HaleInLifeExpectancy.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Location                      72 non-null     object 
 1   Period                        72 non-null     int64  
 2   Indicator                     72 non-null     object 
 3   Dim1                          72 non-null     object 
 4   Hale Expectency               72 non-null     float64
 5   Life expectany                72 non-null     float64
 6   Unnamed: 6                    72 non-null     float64
 7   % OF HALE in life expectancy  72 non-null     int64  
dtypes: float64(3), int64(2), object(3)
memory usage: 4.6+ KB
None

Summary statistics:
            Period  Hale Expectency  Life expectany  Unnamed: 6  \
count    72.000000        72.000000       72.000000   72.000000   
mean   2011.000000        61.403333      

### Merged Mortality

In [4]:
if merged_mortality is not None:
    print("Merged Mortality Data:")
    print(merged_mortality.head())

Merged Mortality Data:
      Location  Period                                          Indicator  \
0  Afghanistan    2019  Infant mortality rate (probability of dying be...   
1  Afghanistan    2019  Infant mortality rate (probability of dying be...   
2  Afghanistan    2019  Infant mortality rate (probability of dying be...   
3  Afghanistan    2018  Infant mortality rate (probability of dying be...   
4  Afghanistan    2018  Infant mortality rate (probability of dying be...   

         Dim1        First Tooltip  
0  Both sexes  46.51 [37.71-55.82]  
1        Male    49.8 [40.2-59.95]  
2      Female  43.05 [34.82-51.96]  
3  Both sexes  48.04 [39.94-56.32]  
4        Male   51.35 [42.56-60.4]  


In [15]:
# Check for missing values in the merged_life_expectancy DataFrame
missing_values_mortality = merged_mortality.isnull()

# Print the DataFrame with missing value indicators
print(missing_values_mortality)


       Location  Period  Indicator   Dim1  First Tooltip
0         False   False      False  False          False
1         False   False      False  False          False
2         False   False      False  False          False
3         False   False      False  False          False
4         False   False      False  False          False
...         ...     ...        ...    ...            ...
77363     False   False      False  False          False
77364     False   False      False  False          False
77365     False   False      False  False          False
77366     False   False      False  False          False
77367     False   False      False  False          False

[77368 rows x 5 columns]


#### Check for Duplicates 


In [35]:
duplicate_rows = merged_mortality[merged_mortality.duplicated()]

# Display duplicate rows (if any)
if not duplicate_rows.empty:
    print("Duplicate Rows:")
    print(duplicate_rows)
else:
    print("No duplicate rows found.")

No duplicate rows found.


#### Check Unique Values

In [36]:
unique_counts = merged_mortality.nunique()
print(unique_counts)

Location           200
Period              70
Indicator            6
Dim1                 3
First Tooltip    69311
dtype: int64


In [37]:
merged_mortality.describe()

,Period
count,77368.000000
mean,1991.856969
std,18.531457
min,1950.000000
25%,1978.000000
50%,1995.000000
75%,2008.000000
max,2019.000000


In [38]:
print(merged_mortality.dtypes)

Location         object
Period            int64
Indicator        object
Dim1             object
First Tooltip    object
dtype: object


##### location, indicator, dim1, first tooltip are all categorical variables

In [42]:
merged_mortality.head()

,Location,Period,Indicator,Dim1,First Tooltip
0,Afghanistan,2019,Infant mortality rate (probability of dying be...,Both sexes,46.51 [37.71-55.82]
1,Afghanistan,2019,Infant mortality rate (probability of dying be...,Male,49.8 [40.2-59.95]
2,Afghanistan,2019,Infant mortality rate (probability of dying be...,Female,43.05 [34.82-51.96]
3,Afghanistan,2018,Infant mortality rate (probability of dying be...,Both sexes,48.04 [39.94-56.32]
4,Afghanistan,2018,Infant mortality rate (probability of dying be...,Male,51.35 [42.56-60.4]


In [49]:
#Dim1 is irrelevant to our analysis so we will drop it 

# Print the columns in your DataFrame
print(merged_mortality.columns)

# Check if 'Dim1' exists in the columns
if 'Dim1' in merged_mortality.columns:
    # Drop the 'Dim1' column
    merged_mortality.drop(columns=['Dim1'], inplace=True)
else:
    print("'Dim1' column not found in DataFrame.")

# Print the updated DataFrame
merged_mortality.head()


Index(['Location', 'Period', 'Indicator', 'First Tooltip'], dtype='object')
'Dim1' column not found in DataFrame.


,Location,Period,Indicator,First Tooltip
0,Afghanistan,2019,Infant mortality rate (probability of dying be...,46.51 [37.71-55.82]
1,Afghanistan,2019,Infant mortality rate (probability of dying be...,49.8 [40.2-59.95]
2,Afghanistan,2019,Infant mortality rate (probability of dying be...,43.05 [34.82-51.96]
3,Afghanistan,2018,Infant mortality rate (probability of dying be...,48.04 [39.94-56.32]
4,Afghanistan,2018,Infant mortality rate (probability of dying be...,51.35 [42.56-60.4]


### Merged Life Expectency


In [5]:
if merged_life_expectancy is not None:
    print("Merged Life Expectancy Data:")
    print(merged_life_expectancy.head())

Merged Life Expectancy Data:
      Location  Period                         Indicator        Dim1  \
0  Afghanistan    2019  Life expectancy at birth (years)  Both sexes   
1  Afghanistan    2019  Life expectancy at birth (years)        Male   
2  Afghanistan    2019  Life expectancy at birth (years)      Female   
3  Afghanistan    2015  Life expectancy at birth (years)  Both sexes   
4  Afghanistan    2015  Life expectancy at birth (years)        Male   

   First Tooltip  Hale Expectency  Life expectany  Unnamed: 6  \
0          63.21              NaN             NaN         NaN   
1          63.29              NaN             NaN         NaN   
2          63.16              NaN             NaN         NaN   
3          61.65              NaN             NaN         NaN   
4          61.04              NaN             NaN         NaN   

   % OF HALE in life expectancy  
0                           NaN  
1                           NaN  
2                           NaN  
3          

In [6]:
# Check the columns before dropping
print("Columns before dropping:")
print(merged_life_expectancy.columns)

# Drop the 'Unnamed: 6' column
merged_life_expectancy.drop(columns=['Unnamed: 6'], inplace=True)

# Check the columns after dropping
print("\nColumns after dropping 'Unnamed: 6':")
print(merged_life_expectancy.columns)


Columns before dropping:
Index(['Location', 'Period', 'Indicator', 'Dim1', 'First Tooltip',
       'Hale Expectency', 'Life expectany', 'Unnamed: 6',
       '% OF HALE in life expectancy'],
      dtype='object')

Columns after dropping 'Unnamed: 6':
Index(['Location', 'Period', 'Indicator', 'Dim1', 'First Tooltip',
       'Hale Expectency', 'Life expectany', '% OF HALE in life expectancy'],
      dtype='object')


In [13]:
merged_life_expectancy.tail()

,Location,Period,Indicator,Dim1,First Tooltip,Hale Expectency,Life expectany,% OF HALE in life expectancy
4604,Western Pacific,2010,Healthy life expectancy (HALE) at birth (years),Male,NaN,65.36,72.52,90.0
4605,Western Pacific,2010,Healthy life expectancy (HALE) at birth (years),Female,NaN,68.79,78.75,87.0
4606,Western Pacific,2000,Healthy life expectancy (HALE) at birth (years),Both sexes,NaN,64.27,72.42,89.0
4607,Western Pacific,2000,Healthy life expectancy (HALE) at birth (years),Male,NaN,62.91,69.80,90.0
4608,Western Pacific,2000,Healthy life expectancy (HALE) at birth (years),Female,NaN,65.78,75.33,87.0


In [14]:
# Check for missing values in the merged_life_expectancy DataFrame
missing_values_life_expectancy = merged_life_expectancy.isnull()

# Print the DataFrame with missing value indicators
print(missing_values_life_expectancy)


      Location  Period  Indicator   Dim1  First Tooltip  Hale Expectency  \
0        False   False      False  False          False             True   
1        False   False      False  False          False             True   
2        False   False      False  False          False             True   
3        False   False      False  False          False             True   
4        False   False      False  False          False             True   
...        ...     ...        ...    ...            ...              ...   
4604     False   False      False  False           True            False   
4605     False   False      False  False           True            False   
4606     False   False      False  False           True            False   
4607     False   False      False  False           True            False   
4608     False   False      False  False           True            False   

      Life expectany  % OF HALE in life expectancy  
0               True              

In [16]:
# true values = null 
#since they are important to our analysis will use imputation 
#to decide on whether to use mean,mode, median
merged_life_expectancy.describe()

,Period,First Tooltip,Hale Expectency,Life expectany,% OF HALE in life expectancy
count,4609.000000,4465.000000,144.000000,144.000000,144.000000
mean,2010.980256,65.888116,61.403333,70.556806,87.013889
std,7.231646,9.410087,5.961208,6.856057,1.404220
min,1920.000000,25.370000,45.030000,51.060000,85.000000
25%,2000.000000,60.140000,57.870000,66.585000,86.000000
50%,2010.000000,66.400000,61.665000,71.375000,87.000000
75%,2015.000000,72.280000,66.360000,75.525000,88.000000
max,2019.000000,86.940000,70.240000,81.290000,90.000000


In [22]:
print(merged_life_expectancy.dtypes)
#Mean: Use the mean when the data is normally distributed and does not have outliers.
#Median: Use the median when the data has outliers or is not normally distributed.
#Mode: Use the mode for categorical or discrete data, although in your case, you're dealing with continuous variables.

Location                         object
Period                            int64
Indicator                        object
Dim1                             object
First Tooltip                   float64
Hale Expectency                 float64
Life expectany                  float64
% OF HALE in life expectancy    float64
dtype: object


In [25]:
#location, indicator, dim1 are categorical variables
#For continuous variables Fist Tooltip, Hale Expectancy, Life Expectancy,and % OF HALE in life expectancy
#use the median for imputation due to the presence of outliers, which can skew the mean.
# Calculate the medians
median_hale = merged_life_expectancy['Hale Expectency'].median()
median_life = merged_life_expectancy['Life expectany'].median()
median_hale_pct = merged_life_expectancy['% OF HALE in life expectancy'].median()
median_tooltip = merged_life_expectancy['First Tooltip'].median()

# Fill missing values with medians
merged_life_expectancy['Hale Expectency'].fillna(median_hale, inplace=True)
merged_life_expectancy['Life expectany'].fillna(median_life, inplace=True)
merged_life_expectancy['% OF HALE in life expectancy'].fillna(median_hale_pct, inplace=True)
merged_life_expectancy['First Tooltip'].fillna(median_tooltip, inplace=True)

# Verify that missing values are filled
print(merged_life_expectancy.isnull().sum())


Location                        0
Period                          0
Indicator                       0
Dim1                            0
First Tooltip                   0
Hale Expectency                 0
Life expectany                  0
% OF HALE in life expectancy    0
dtype: int64


C:\Users\user\AppData\Local\Temp\ipykernel_13684\2695820863.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_life_expectancy['Hale Expectency'].fillna(median_hale, inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_13684\2695820863.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always 

In [50]:
merged_life_expectancy.head()

,Location,Period,Indicator,Dim1,First Tooltip,Hale Expectency,Life expectany,% OF HALE in life expectancy
0,Afghanistan,2019,Life expectancy at birth (years),Both sexes,63.21,61.665,71.375,87.0
1,Afghanistan,2019,Life expectancy at birth (years),Male,63.29,61.665,71.375,87.0
2,Afghanistan,2019,Life expectancy at birth (years),Female,63.16,61.665,71.375,87.0
3,Afghanistan,2015,Life expectancy at birth (years),Both sexes,61.65,61.665,71.375,87.0
4,Afghanistan,2015,Life expectancy at birth (years),Male,61.04,61.665,71.375,87.0


In [51]:
print(merged_life_expectancy.dtypes)

Location                         object
Period                            int64
Indicator                        object
Dim1                             object
First Tooltip                   float64
Hale Expectency                 float64
Life expectany                  float64
% OF HALE in life expectancy    float64
dtype: object


In [55]:

#Dim1 is irrelevant to our analysis so we will drop it 

# Print the columns in your DataFrame
print(merged_life_expectancy.columns)

# Check if 'Dim1' exists in the columns
if 'Dim1' in merged_life_expectancy.columns:
    # Drop the 'Dim1' column
    merged_life_expectancy.drop(columns=['Dim1'], inplace=True)
else:
    print("'Dim1' column not found in DataFrame.")

# Print the updated DataFrame
merged_life_expectancy.head()

Index(['Location', 'Period', 'Indicator', 'Dim1', 'First Tooltip',
       'Hale Expectency', 'Life expectany', '% OF HALE in life expectancy'],
      dtype='object')


,Location,Period,Indicator,First Tooltip,Hale Expectency,Life expectany,% OF HALE in life expectancy
0,Afghanistan,2019,Life expectancy at birth (years),63.21,61.665,71.375,87.0
1,Afghanistan,2019,Life expectancy at birth (years),63.29,61.665,71.375,87.0
2,Afghanistan,2019,Life expectancy at birth (years),63.16,61.665,71.375,87.0
3,Afghanistan,2015,Life expectancy at birth (years),61.65,61.665,71.375,87.0
4,Afghanistan,2015,Life expectancy at birth (years),61.04,61.665,71.375,87.0


#### Checking for Duplicate Rows

In [56]:
duplicate_rows = merged_life_expectancy[merged_life_expectancy.duplicated()]

# Display duplicate rows (if any)
if not duplicate_rows.empty:
    print("Duplicate Rows:")
    print(duplicate_rows)
else:
    print("No duplicate rows found.")

Duplicate Rows:
             Location  Period  \
3405           Kuwait    2010   
3795             Oman    2019   
3809         Pakistan    2015   
4537           Africa    2019   
4538           Africa    2019   
...               ...     ...   
4604  Western Pacific    2010   
4605  Western Pacific    2010   
4606  Western Pacific    2000   
4607  Western Pacific    2000   
4608  Western Pacific    2000   

                                            Indicator  First Tooltip  \
3405  Healthy life expectancy (HALE) at birth (years)          69.21   
3795  Healthy life expectancy (HALE) at birth (years)          64.51   
3809  Healthy life expectancy (HALE) at birth (years)          55.78   
4537  Healthy life expectancy (HALE) at birth (years)          66.40   
4538  Healthy life expectancy (HALE) at birth (years)          66.40   
...                                               ...            ...   
4604  Healthy life expectancy (HALE) at birth (years)          66.40   
4605  Healt

### Merged Health Services

In [7]:
if merged_health_services is not None:
    print("Merged Health Services Data:")
    print(merged_health_services.head())

Merged Health Services Data:
      Location  Period                                          Indicator  \
0  Afghanistan    2017  Population using at least basic drinking-water...   
1  Afghanistan    2016  Population using at least basic drinking-water...   
2  Afghanistan    2015  Population using at least basic drinking-water...   
3  Afghanistan    2014  Population using at least basic drinking-water...   
4  Afghanistan    2013  Population using at least basic drinking-water...   

   First Tooltip Dim1  
0          57.32  NaN  
1          54.84  NaN  
2          52.39  NaN  
3          49.96  NaN  
4          47.56  NaN  


In [27]:
# Check for missing values in the merged_life_expectancy DataFrame
missing_values_merged_health_services = merged_health_services.isnull()

# Print the DataFrame with missing value indicators
print(missing_values_merged_health_services)


       Location  Period  Indicator  First Tooltip  Dim1
0         False   False      False          False  True
1         False   False      False          False  True
2         False   False      False          False  True
3         False   False      False          False  True
4         False   False      False          False  True
...         ...     ...        ...            ...   ...
20954     False   False      False          False  True
20955     False   False      False          False  True
20956     False   False      False          False  True
20957     False   False      False          False  True
20958     False   False      False          False  True

[20959 rows x 5 columns]


In [29]:
print(merged_health_services.dtypes)

Location          object
Period             int64
Indicator         object
First Tooltip    float64
Dim1              object
dtype: object


In [31]:
#location, indicator, dim1 are categorical variables
merged_health_services.describe()

,Period,First Tooltip
count,20959.000000,20959.000000
mean,2008.994275,68.297133
std,5.129152,31.351259
min,2000.000000,0.000000
25%,2005.000000,41.675000
50%,2009.000000,80.950000
75%,2013.000000,96.800000
max,2019.000000,100.000000


In [34]:
#Dim1 is a categorical variable so use mode imputation
# Calculate the mode for 'Dim1' column
mode_value = merged_health_services['Dim1'].mode()[0]  # Using [0] to get the first mode if multiple

# Fill missing values in 'Dim1' column with the mode
merged_health_services['Dim1'].fillna(mode_value, inplace=True)

# Verify that missing values are filled
print(merged_health_services.isnull().sum())


Location         0
Period           0
Indicator        0
First Tooltip    0
Dim1             0
dtype: int64


C:\Users\user\AppData\Local\Temp\ipykernel_13684\2069298338.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_health_services['Dim1'].fillna(mode_value, inplace=True)


In [52]:
merged_health_services.head()

,Location,Period,Indicator,First Tooltip,Dim1
0,Afghanistan,2017,Population using at least basic drinking-water...,57.32,Total
1,Afghanistan,2016,Population using at least basic drinking-water...,54.84,Total
2,Afghanistan,2015,Population using at least basic drinking-water...,52.39,Total
3,Afghanistan,2014,Population using at least basic drinking-water...,49.96,Total
4,Afghanistan,2013,Population using at least basic drinking-water...,47.56,Total


### Merged Disease Incidence

In [8]:
if merged_disease_incidence is not None:
    print("Merged Disease Incidence Data:")
    print(merged_disease_incidence.head())

Merged Disease Incidence Data:
      Location                                          Indicator  Period  \
0  Afghanistan  Reported number of people requiring interventi...    2018   
1  Afghanistan  Reported number of people requiring interventi...    2017   
2  Afghanistan  Reported number of people requiring interventi...    2016   
3  Afghanistan  Reported number of people requiring interventi...    2015   
4  Afghanistan  Reported number of people requiring interventi...    2014   

  First Tooltip Dim1  
0      14834701  NaN  
1      14786722  NaN  
2      13275429  NaN  
3      13765392  NaN  
4      13384908  NaN  


### Maternal Health

In [10]:
if merged_maternal_health is not None:
    print("Merged Maternal Health Data:")
    print(merged_maternal_health.head())

Merged Maternal Health Data:
      Location  Period                                          Indicator  \
0  Afghanistan    2017  Maternal mortality ratio (per 100 000 live bir...   
1  Afghanistan    2016  Maternal mortality ratio (per 100 000 live bir...   
2  Afghanistan    2015  Maternal mortality ratio (per 100 000 live bir...   
3  Afghanistan    2014  Maternal mortality ratio (per 100 000 live bir...   
4  Afghanistan    2013  Maternal mortality ratio (per 100 000 live bir...   

    First Tooltip  
0  638 [427-1010]  
1  673 [457-1040]  
2  701 [501-1020]  
3  786 [592-1080]  
4  810 [617-1080]  
